In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time

In [62]:
a = np.array([1, 0, 3])
b = np.zeros((3, 4))
b[np.arange(3), a] = 1
# https://stackoverflow.com/questions/29831489/convert-array-of-indices-to-1-hot-encoded-numpy-array

In [88]:
# Langsamer sobald grösserer Array z.B. [1,3,45,64,64]
A = torch.randn(1,3,45,64,64)
B = A.clone()
start_time = time.time() 
A_max, A_index = torch.max(A,2,keepdim=True)
A[A<A_max] = 0
end_time = time.time()
print(end_time-start_time)
# print(B)

0.3170180320739746


In [89]:
# K = torch.rand(1,2,3,4,4)
start_time = time.time()
batch_size, number_anchors, number_classes, x, y = B.shape
K_max, K_index = torch.max(B,2)# max finden
batch = np.zeros(number_anchors*x*y) # batch bestimmen
# print('batch: ', batch.size())
x_index = np.arange(x).tolist() # tolist, weil numpy extend nicht kennt
x_index.extend(x_index*(number_anchors*x-1)) # mit extend kann mehrmals die gleiche Liste angehängt werden
# print('x_index: ', x_index)

y_index = np.repeat(np.arange(y),y,axis=0).tolist() # reapet => 0123 wird 00112233
y_index.extend(y_index*(number_anchors-1))
# print('y_index: ', y_index)


K_index_new = K_index.view(1,number_anchors*x*y)
# print('K_index_new size: ', K_index_new)

number_anchors_index = np.repeat(np.arange(number_anchors),x*y,axis=0)
# print('number_anchorse: ', number_anchors_index)


# print(K)
# print(K_index)
K_new = torch.zeros(batch_size, number_anchors, number_classes, x, y)

K_new[batch, number_anchors_index, K_index_new, np.asarray(y_index), np.asarray(x_index)] = 1 # muss array sein keine Liste => np.asarray
# print(K_new)
K_max_finish = torch.mul(K_new,B)
end_time = time.time()
print(end_time-start_time)

0.026001691818237305


In [90]:
A = K_max_finish.clone()
B = K_max_finish.clone()
threshold = 0.5

In [91]:
start_time = time.time()

A[A<threshold] = 0

end_time = time.time()
print(end_time-start_time)

0.29701685905456543


In [92]:
start_time = time.time()

K_max_end = B[batch, number_anchors_index, K_index_new, np.asarray(y_index), np.asarray(x_index)]
K_max_end[K_max_end<threshold] = 0
K_thres = torch.zeros(batch_size, number_anchors, number_classes, x, y)
K_thres[batch, number_anchors_index, K_index_new, np.asarray(y_index), np.asarray(x_index)] = K_max_end

end_time = time.time()
print(end_time-start_time)

0.02500128746032715


In [86]:
print(B)
print(K_thres)

tensor([[[[[-0.0000,  0.0000,  0.5743],
           [-0.0000, -0.0000,  0.0000],
           [-0.0000,  0.4487,  1.7352]],

          [[ 0.8342,  1.4084,  0.0000],
           [ 1.8344,  0.9143,  0.7073],
           [ 1.2349,  0.0000, -0.0000]]],


         [[[ 0.0000, -0.0000, -0.0275],
           [-0.0000, -0.0000,  0.0000],
           [-0.0000, -0.0000, -0.0000]],

          [[ 0.9682, -0.4264, -0.0000],
           [ 1.0909,  0.7947,  1.1667],
           [ 1.0717,  0.2389,  0.5408]]]]])
tensor([[[[[ 0.0000,  0.0000,  0.5743],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  1.7352]],

          [[ 0.8342,  1.4084,  0.0000],
           [ 1.8344,  0.9143,  0.7073],
           [ 1.2349,  0.0000,  0.0000]]],


         [[[ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000]],

          [[ 0.9682,  0.0000,  0.0000],
           [ 1.0909,  0.7947,  1.1667],
           [ 1.0717,  0.0000,  0.5408]]]]])


In [87]:
print(torch.eq(B,K_thres))

tensor([[[[[ 1,  1,  1],
           [ 1,  1,  1],
           [ 1,  0,  1]],

          [[ 1,  1,  1],
           [ 1,  1,  1],
           [ 1,  1,  1]]],


         [[[ 1,  1,  0],
           [ 1,  1,  1],
           [ 1,  1,  1]],

          [[ 1,  0,  1],
           [ 1,  1,  1],
           [ 1,  0,  1]]]]], dtype=torch.uint8)
